In [1]:
import pandas as pd

In [15]:
csvfile = "/Users/alifabdullah/Collaboration/Kaggle-ML-Algorithm-Musings/datasets/diabetes.csv"
csv_in_panda_form = pd.read_csv(csvfile)
target_column = "Outcome"
feature_columns = [feature_header for feature_header in csv_in_panda_form.columns.drop(target_column)]

#print(csv_in_panda_form)
#print(feature_columns)

#for row in csv_in_panda_form.iterrows():
#  print(row[1]["Outcome"])

print(len(csv_in_panda_form[csv_in_panda_form["Pregnancies"] == 0]))
print((csv_in_panda_form["Pregnancies"].unique()))
  


111
[ 6  1  8  0  5  3 10  2  4  7  9 11 13 15 17 12 14]


In [ ]:
def single_feature_split_error_calculator(dataframe, feature):
  pass

def dataset_consistency_metrics(dataframe, list_of_labels, target_column):
  """
  Tells you the counts of instances, in a dataset, corresponding to labels, as 
  well as the error of the dataset, where the error is calculated as the number
  of dataset instances corresponding to the label with the least instances.

  Args:
    dataframe - A Pandas DataFrame, representing our dataset
    list_of_labels - List of labels that represents all labels that could be assigned (replaced by self.list_of_labels going forward)
    target_column: The name of the column we are trying to predict (replaced by self.target_column going forward)
  
  Returns:
    error - The error, as defined above
    label_counts - A dictionary mapping between labels and their respective counts
  """
  label_count_mapping_dict = dict()
  for key in list_of_labels:
    label_count_mapping_dict[key] = 0
  
  for row in dataframe.iterrows():
    label_count_mapping_dict[row[1][target_column]] += 1 
  
  error = float("inf")
  for key in list_of_labels:
    if label_count_mapping_dict[key] < error:
      error = label_count_mapping_dict[key]
  return error, label_count_mapping_dict

dataset_consistency_metrics(csv_in_panda_form, [0,1], "Outcome")

(268, {0: 500, 1: 268})